In [17]:
import pandas as pd
import numpy as np
from script import *
from feature_engineering import *
import db_operations as dbop
from constants import *
import data_process as dp
    
import itertools
import lightgbm as lgbm
import matplotlib.pyplot as plt


import time

In [18]:
pd.set_option('display.max_rows',200)
idx = pd.IndexSlice
agg_operations =['mean','median','max','min','std','size']

In [19]:
start = 20100101
cursor = dbop.connect_db("sqlite3").cursor()

df_d = dbop.create_df(cursor, STOCK_DAY[TABLE], start=start)
df_d = dp.proc_stock_d(dp.prepare_stock_d(df_d))
df_d.drop(columns=['open0','high0','low0','vol0'],inplace=True)
df_d['pct'] = df_d.sort_index().groupby('code')['close'].pct_change()*100
df_d['amt']/=1e5
df_d = df_d.astype('float32')
print(df_d.columns)
print(df_d.shape)
df_d.info(memory_usage='deep')

select * from stock_day where date>=20100101
Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct'],
      dtype='object')
(3064850, 11)
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3064850 entries, (000001.SZ, 2010-01-04 00:00:00) to (603997.SH, 2020-04-09 00:00:00)
Data columns (total 11 columns):
open          float32
high          float32
low           float32
close         float32
vol           float32
amt           float32
adj_factor    float32
avg           float32
close0        float32
avg0          float32
pct           float32
dtypes: float32(11)
memory usage: 220.4 MB


In [20]:
start_date = '-'.join([str(start)[:4],str(start)[4:6],str(start)[6:8]])
df_r = pd.read_parquet(r"database\return_5%_10%_20_8_15%").loc[idx[:,start_date:],:].sort_index()

In [23]:
df = df_d
periods = [5,10,20,30,60,120,250]
for k in periods:
    df['{:d}ma'.format(k)]=df.reset_index('code').groupby('code')['close'].rolling(k).mean()
print(df.columns)
    
for col2 in ["{}ma".format(k) for k in periods]:
    col1 = 'close'
    df['{}/{}'.format(col1,col2)] = df[col1]/df[col2]
print(df.columns)

Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct', '5ma', '10ma', '20ma', '30ma', '60ma', '120ma',
       '250ma'],
      dtype='object')
Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct', '5ma', '10ma', '20ma', '30ma', '60ma', '120ma',
       '250ma', 'close/5ma', 'close/10ma', 'close/20ma', 'close/30ma',
       'close/60ma', 'close/120ma', 'close/250ma'],
      dtype='object')


In [24]:
print(df.index.is_lexsorted())
df.reset_index('code').groupby('code').expanding(1).agg(['count','quantile'])

True


open   high    low  close    vol    amt  \
                                count  count  count  count  count  count   
code      code      date                                                   
000001.SZ 000001.SZ 2010-01-04    1.0    1.0    1.0    1.0    1.0    1.0   
                    2010-01-05    2.0    2.0    2.0    2.0    2.0    2.0   
                    2010-01-06    3.0    3.0    3.0    3.0    3.0    3.0   
                    2010-01-07    4.0    4.0    4.0    4.0    4.0    4.0   
                    2010-01-08    5.0    5.0    5.0    5.0    5.0    5.0   
...                               ...    ...    ...    ...    ...    ...   
                    2014-02-18  927.0  927.0  927.0  927.0  927.0  927.0   
                    2014-02-19  928.0  928.0  928.0  928.0  928.0  928.0   
                    2014-02-20  929.0  929.0  929.0  929.0  929.0  929.0   
                    2014-02-21  930.0  930.0  930.0  930.0  930.0  930.0   
                    2014-02-24  931.0  931.0  931.0  931.0  931.0  931.0   

                               adj_factor    avg close0   avg0  ...   60ma  \
                                    count  count  count  count  ...  count   
code      code      date                                        ...          
000001.SZ 000001.SZ 2010-01-04        1.0    1.0    1.0    1.0  ...    0.0   
                    2010-01-05        2.0    2.0    2.0    2.0  ...    0.0   
                    2010-01-06        3.0    3.0    3.0    3.0  ...    0.0   
                    2010-01-07        4.0    4.0    4.0    4.0  ...    0.0   
                    2010-01-08        5.0    5.0    5.0    5.0  ...    0.0   
...                                   ...    ...    ...    ...  ...    ...   
                    2014-02-18      996.0  927.0  927.0  927.0  ...  351.0   
                    2014-02-19      997.0  928.0  928.0  928.0  ...  352.0   
                    2014-02-20      998.0  929.0  929.0  929.0  ...  353.0   
                    2014-02-21      999.0  930.0  930.0  930.0  ...  354.0   
                    2014-02-24     1000.0  931.0  931.0  931.0  ...  355.0   

                                120ma 250ma close/5ma close/10ma close/20ma  \
                                count count     count      count      count   
code      code      date                                                      
000001.SZ 000001.SZ 2010-01-04    0.0   0.0       0.0        0.0        0.0   
                    2010-01-05    0.0   0.0       0.0        0.0        0.0   
                    2010-01-06    0.0   0.0       0.0        0.0        0.0   
                    2010-01-07    0.0   0.0       0.0        0.0        0.0   
                    2010-01-08    0.0   0.0       1.0        0.0        0.0   
...                               ...   ...       ...        ...        ...   
                    2014-02-18  195.0  65.0     867.0      794.0      684.0   
                    2014-02-19  195.0  65.0     868.0      795.0      685.0   
                    2014-02-20  195.0  65.0     869.0      796.0      686.0   
                    2014-02-21  195.0  65.0     870.0      797.0      687.0   
                    2014-02-24  195.0  65.0     871.0      798.0      688.0   

                               close/30ma close/60ma close/120ma close/250ma  
                                    count      count       count       count  
code      code      date                                                      
000001.SZ 000001.SZ 2010-01-04        0.0        0.0         0.0         0.0  
                    2010-01-05        0.0        0.0         0.0         0.0  
                    2010-01-06        0.0        0.0         0.0         0.0  
                    2010-01-07        0.0        0.0         0.0         0.0  
                    2010-01-08        0.0        0.0         0.0         0.0  
...                                   ...        ...         ...         ...  
                    2014-02-18      591.0      351.0     